# ML-LinearReg-02
## devbuildit.com for more details

In [ ]:
# Import Torch
import torch
from torch import nn

# Matplotlib for visualisations
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
print(f"Pytorch version: {torch.__version__}")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device set to: {device}")

In [3]:
slope = 0.8
intercept =0.5

start =0
end =1
step =0.02

X = torch.arange(start=start,end=end,step=step).unsqueeze(dim=1)
y=slope*X + intercept

# split data (80% training, 20% test)
train_split = int(0.8 * len(X))
X_train, y_train = X[:train_split], y[:train_split]
X_test, y_test = X[train_split:], y[train_split:]

In [4]:
def plot_predictions(train_data=X_train,
                     train_labels=y_train,
                     test_data=X_test,
                     test_labels=y_test,
                     inference=None):

    plt.figure(figsize=(12,7))
    plt.scatter(train_data, train_labels, c="b", s=4, label="Training data")
    plt.scatter(test_data, test_labels, c="c", s=4, label="Testing data")
    if inference is not None:
        plt.scatter(test_data, inference, c="r", s=4, label="Model Inference")
    plt.legend(prop={"size":10})

In [ ]:
plot_predictions()

In [ ]:
class LinearRegresssionModel(nn.Module):       
        super().__init__()
        self.slope = nn.Parameter(torch.randn(1, 
                                                requires_grad=True,  
                                                device=device,
                                                dtype=float))
        self.intercept = nn.Parameter(torch.randn(1, 
                                                requires_grad=True,  
                                                device=device,
                                                dtype=float))

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.slope * x + self.intercept

In [ ]:
# create random seed for repeatablity
torch.manual_seed(99)

#instantiate new model object
model_02 = LinearRegresssionModel()

#Observe model initial parameters
model_02.state_dict()

In [8]:
model_02.eval()
with torch.inference_mode():
    y_preds = model_02(X_test)

In [ ]:
plot_predictions(inference=y_preds)

In [ ]:
## Train the model

# Loss - mean absolute error
loss_fn = nn.L1Loss()

# Optimiziser 
optimizer = torch.optim.SGD(params=model_02.parameters(),
                            lr=0.01)

slope_history = []
intercept_history = []
loss_history = []

epochs = 350 

# Training and testing loops
for epoch in range(epochs):
    model_02.train()
    y_pred = model_02(X_train)
    loss = loss_fn(y_pred, y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch % 50 == 0: 
        print(f"Epoch: {epoch:03}/{epochs} | Loss: {loss:.6f} | Slope: {model_02.slope.detach().clone().numpy()} | Intercept: {model_02.intercept.detach().clone().numpy()}")    

    loss_history.append(loss.item())
    slope_history.append(model_02.slope.detach().clone().numpy())
    intercept_history.append(model_02.intercept.detach().clone().numpy())

In [ ]:
# Convert lists to numpy arrays for easier plotting
weight_history = np.array(slope_history)
bias_history = np.array(intercept_history)
epochs_range = np.array(epochs_range)

plt.figure(figsize=(12, 8))

# Plot 1: Model Slope valuesW
plt.subplot(1, 3, 1)
plt.plot(epochs_range, slope_history, marker='o')
plt.title('Calculated slope per epoch')
plt.xlabel('Epoch')
plt.ylabel('Slope Value')
plt.grid(True)

# Plot 2: Model Intercept values
plt.subplot(1, 3, 2)
plt.plot(epochs_range, intercept_history, marker='o', color='orange')
plt.title('Calculated intercept per epoch')
plt.xlabel('Epoch')
plt.ylabel('Intercept Value')
plt.grid(True)

# Plot 3: Loss over epochs
plt.subplot(1, 3, 3)
plt.plot(epochs_range, loss_history, marker='x', color='green')
plt.title('Loss per epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
model_02.eval()
with torch.inference_mode():
    y_model_eval = model_02(X_test)
plot_predictions(inference=y_model_eval)